<a href="https://colab.research.google.com/github/geopan2000/Exploring-Mental-Health-Data/blob/main/Mental-Health-Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
# Clone the GitHub repository
!git clone https://github.com/geopan2000/Exploring-Mental-Health-Data.git

# Change directory to the cloned repository
%cd Exploring-Mental-Health-Data

Cloning into 'Exploring-Mental-Health-Data'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 5.28 MiB | 3.41 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data/Exploring-Mental-Health-Data


In [87]:
import pandas as pd

In [88]:
train_set = pd.read_csv('/content/Exploring-Mental-Health-Data/data/train.csv')
test_set = pd.read_csv('/content/Exploring-Mental-Health-Data/data/test.csv')

In [89]:
test_set.head()

,id,Name,Gender,Age,City,Working Professional or Student,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,140700,Shivam,Male,53.0,Visakhapatnam,Working Professional,Judge,NaN,2.0,NaN,NaN,5.0,Less than 5 hours,Moderate,LLB,No,9.0,3.0,Yes
1,140701,Sanya,Female,58.0,Kolkata,Working Professional,Educational Consultant,NaN,2.0,NaN,NaN,4.0,Less than 5 hours,Moderate,B.Ed,No,6.0,4.0,No
2,140702,Yash,Male,53.0,Jaipur,Working Professional,Teacher,NaN,4.0,NaN,NaN,1.0,7-8 hours,Moderate,B.Arch,Yes,12.0,4.0,No
3,140703,Nalini,Female,23.0,Rajkot,Student,NaN,5.0,NaN,6.84,1.0,NaN,More than 8 hours,Moderate,BSc,Yes,10.0,4.0,No
4,140704,Shaurya,Male,47.0,Kalyan,Working Professional,Teacher,NaN,5.0,NaN,NaN,5.0,7-8 hours,Moderate,BCA,Yes,3.0,4.0,No


In [90]:
# Count rows where both columns are NaN
both_nan_count = ((train_set['Job Satisfaction'].isna()) & (train_set['Study Satisfaction'].isna())).sum()

# Count rows where both columns have values
both_filled_count = ((train_set['Job Satisfaction'].notna()) & (train_set['Study Satisfaction'].notna())).sum()

print(f"Rows where both columns are NaN: {both_nan_count}")
print(f"Rows where both columns have values: {both_filled_count}")

Rows where both columns are NaN: 15
Rows where both columns have values: 2


Name is clearly a not realted factor for determining if one has depression or not thus, will be dropped.

In [93]:
print(test_set.isna().sum())
# train_set.describe()
# train_set.info()

for col in test_set.columns:
  print(col, test_set[col].nunique())

id                                           0
Name                                         0
Gender                                       0
Age                                          0
City                                         0
Working Professional or Student              0
Profession                               24632
Academic Pressure                        75033
Work Pressure                            18778
CGPA                                     75034
Study Satisfaction                       75033
Job Satisfaction                         18774
Sleep Duration                               0
Dietary Habits                               5
Degree                                       2
Have you ever had suicidal thoughts ?        0
Work/Study Hours                             0
Financial Stress                             0
Family History of Mental Illness             0
dtype: int64
id 93800
Name 374
Gender 2
Age 44
City 68
Working Professional or Student 2
Profession 64
Acad

CGPA 331
Sleep Duration 36
Dietary Habits 23







In [85]:
train_set.drop(columns=['Name', 'id'], inplace=True)
train_set['Age'] = train_set['Age'] / 100 #Normalized by the Max value
train_set['Age'] = train_set['Age'].fillna(train_set['Age'].mean())

In [56]:
train_set['Gender'] = train_set['Gender'].map({'Male' : 1, 'Female' : 0}) #Always trasnforming 2 option object types into binary



train_set['Working Professional or Student'] = train_set['Working Professional or Student'].map({'Working Professional' : 1, 'Student' : 0}) #Always trasnforming 2 option object types into binary
train_set['Family History of Mental Illness'].map({'Yes' : 1, 'No' : 0}) #Always trasnforming 2 option object types into binary
train_set['Have you ever had suicidal thoughts ?'] = train_set['Have you ever had suicidal thoughts ?'].map({'Yes' : 1, 'No' : 0}) #Always trasnforming 2 option object types into binary
train_set['Work/Study Hours'] = train_set['Work/Study Hours'] / 24 #Normalized by the Max value
train_set['Financial Stress'] = train_set['Financial Stress'] / 5 #Normalized by the Max value
train_set['City'] = train_set['City'].map(train_set['City'].value_counts()) # frequency encoding, 98 different cities
train_set['Degree'] = train_set['Degree'].map(train_set['Degree'].value_counts()) # frequency encoding
train_set['Profession'] = train_set['Profession'].map(train_set['Profession'].value_counts()) # frequency encoding

# Create the new feature 'Study/Job Satisfaction'
train_set['Study/Job Satisfaction'] = train_set.apply(
    lambda row: row['Study Satisfaction'] if pd.notna(row['Study Satisfaction']) and pd.isna(row['Job Satisfaction']) else
                row['Job Satisfaction'] if pd.isna(row['Study Satisfaction']) and pd.notna(row['Job Satisfaction']) else
                (row['Study Satisfaction'] + row['Job Satisfaction']) / 2 if pd.notna(row['Study Satisfaction']) and pd.notna(row['Job Satisfaction']) else
                None,  # Placeholder for rows where both are NaN
    axis=1
)

# Fill any remaining NaN in 'Study/Job Satisfaction' with its mean
train_set['Study/Job Satisfaction'] = train_set['Study/Job Satisfaction'].fillna(train_set['Study/Job Satisfaction'].mean())

# Create the new feature 'Academic/Work Pressure'
train_set['Academic/Work Pressure'] = train_set.apply(
    lambda row: row['Academic Pressure'] if pd.notna(row['Academic Pressure']) and pd.isna(row['Work Pressure']) else
                row['Work Pressure'] if pd.isna(row['Academic Pressure']) and pd.notna(row['Work Pressure']) else
                (row['Academic Pressure'] + row['Work Pressure']) / 2 if pd.notna(row['Academic Pressure']) and pd.notna(row['Work Pressure']) else
                None,  # Placeholder for rows where both are NaN
    axis=1
)

# Fill any remaining NaN in 'Academic/Work Pressure' with its mean
train_set['Academic/Work Pressure'] = train_set['Academic/Work Pressure'].fillna(train_set['Academic/Work Pressure'].mean())

In [57]:
train_set['Working Professional or Student'].value_counts()

,count
Working Professional or Student,
1,112799
0,27901


In [64]:
test_set = pd.read_csv('/content/Exploring-Mental-Health-Data/data/test.csv')
test_set['Working Professional or Student'] = test_set['Working Professional or Student'].map({'Working Professional' : 1, 'Student' : 0}) #Always trasnforming 2 option object types into binary
print(test_set['Working Professional or Student'].head())
test_set['Working Professional or Student'].value_counts()

0    1
1    1
2    1
3    0
4    1
Name: Working Professional or Student, dtype: int64


,count
Working Professional or Student,
1,75028
0,18772
